# 스테이블 디퓨전 XL

[스테이블 디퓨전 XL](https://huggingface.co/papers/2307.01952) (SDXL)은 이전 스테이블 디퓨전 모델을 세 가지 주요 방식으로 반복하는 강력한 텍스트-이미지 생성 모델입니다.

1. UNet이 3배 더 크고 SDXL은 두 번째 텍스트 인코더(OpenCLIP ViT-bigG/14)를 원래 텍스트 인코더와 결합하여 매개변수 수를 크게 늘립니다.
2. 훈련 데이터가 삭제되는 것을 방지하고 생성된 이미지를 자르는 방법에 대한 더 많은 제어권을 얻기 위해 크기 및 자르기 조정을 도입합니다.
3. 2단계 모델 프로세스를 도입합니다. *기본* 모델(독립 실행형 모델로도 실행 가능)은 추가적인 고품질 세부 정보를 추가하는 *정제기* 모델에 대한 입력으로 이미지를 생성합니다.

이 가이드에서는 텍스트-이미지, 이미지-이미지 및 인페인팅에 SDXL을 사용하는 방법을 보여줍니다.

시작하기 전에 다음 라이브러리가 설치되어 있는지 확인하십시오.

In [ ]:
# Colab에서 필요한 라이브러리를 설치하려면 주석 처리를 제거하십시오.
#!pip install -q diffusers transformers accelerate invisible-watermark>=0.2.0

<Tip warning={true}>

생성된 이미지를 식별하는 데 도움이 되도록 [invisible-watermark](https://pypi.org/project/invisible-watermark/) 라이브러리를 설치하는 것이 좋습니다. invisible-watermark 라이브러리가 설치되어 있으면 기본적으로 사용됩니다. 워터마커를 비활성화하려면 다음을 수행하십시오.

In [ ]:
pipeline = StableDiffusionXLPipeline.from_pretrained(..., add_watermarker=False)

</Tip>

## 모델 체크포인트 로드

모델 가중치는 허브 또는 로컬의 별도 하위 폴더에 저장될 수 있으며, 이 경우 [from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained) 메서드를 사용해야 합니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
import torch

pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16"
).to("cuda")

또한 [from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드를 사용하여 허브 또는 로컬에서 단일 파일 형식(`.ckpt` 또는 `.safetensors`)으로 저장된 모델 체크포인트를 로드할 수도 있습니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
import torch

pipeline = StableDiffusionXLPipeline.from_single_file(
    "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0.safetensors",
    torch_dtype=torch.float16
).to("cuda")

refiner = StableDiffusionXLImg2ImgPipeline.from_single_file(
    "https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/blob/main/sd_xl_refiner_1.0.safetensors", torch_dtype=torch.float16
).to("cuda")

## 텍스트-이미지

텍스트-이미지의 경우 텍스트 프롬프트를 전달합니다. 기본적으로 SDXL은 최상의 결과를 위해 1024x1024 이미지를 생성합니다. `height` 및 `width` 매개변수를 768x768 또는 512x512로 설정해 볼 수 있지만 512x512 미만은 작동하지 않을 가능성이 높습니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline_text2image = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

prompt = "정글 속 우주비행사, 차가운 색상 팔레트, 차분한 색상, 디테일, 8k"
image = pipeline_text2image(prompt=prompt).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-text2img.png" alt="정글 속 우주비행사의 생성된 이미지"/>
</div>

## 이미지-이미지

이미지-이미지의 경우 SDXL은 768x768에서 1024x1024 사이의 이미지 크기에서 특히 잘 작동합니다. 초기 이미지를 전달하고 이미지에 조건을 부여할 텍스트 프롬프트를 전달합니다.

In [ ]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid

# 체크포인트를 로드할 때 추가 메모리를 소비하지 않도록 from_pipe 사용
pipeline = AutoPipelineForImage2Image.from_pipe(pipeline_text2image).to("cuda")

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-text2img.png"
init_image = load_image(url)
prompt = "정글에서 프리스비를 잡는 개"
image = pipeline(prompt, image=init_image, strength=0.8, guidance_scale=10.5).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-img2img.png" alt="정글에서 프리스비를 잡는 개의 생성된 이미지"/>
</div>

## 인페인팅

인페인팅의 경우 원본 이미지와 원본 이미지에서 바꾸려는 부분의 마스크가 필요합니다. 마스크된 영역을 무엇으로 바꿀지 설명하는 프롬프트를 만듭니다.

In [ ]:
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

# 체크포인트를 로드할 때 추가 메모리를 소비하지 않도록 from_pipe 사용
pipeline = AutoPipelineForInpainting.from_pipe(pipeline_text2image).to("cuda")

img_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-text2img.png"
mask_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-inpaint-mask.png"

init_image = load_image(img_url)
mask_image = load_image(mask_url)

prompt = "떠다니는 심해 잠수부"
image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, strength=0.85, guidance_scale=12.5).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-inpaint.png" alt="정글 속 심해 잠수부의 생성된 이미지"/>
</div>

## 이미지 품질 개선

SDXL에는 기본 모델에서 고품질 이미지를 생성하기 위해 저노이즈 단계 이미지를 노이즈 제거하는 데 특화된 [정제기 모델](https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0)이 포함되어 있습니다. 정제기를 사용하는 방법에는 두 가지가 있습니다.

1. 기본 모델과 정제기 모델을 함께 사용하여 정제된 이미지를 생성합니다.
2. 기본 모델을 사용하여 이미지를 생성한 다음 정제기 모델을 사용하여 이미지에 세부 정보를 추가합니다(이것이 SDXL이 원래 훈련된 방식입니다).

### 기본 + 정제기 모델

기본 모델과 정제기 모델을 함께 사용하여 이미지를 생성하는 경우 이를 [*전문 노이즈 제거기 앙상블*](https://research.nvidia.com/labs/dir/eDiff-I/)이라고 합니다. 전문가 노이즈 제거기 앙상블 접근 방식은 기본 모델의 출력을 정제기 모델에 전달하는 것보다 전체 노이즈 제거 단계가 적으므로 실행 속도가 훨씬 빨라야 합니다. 그러나 기본 모델의 출력에는 여전히 많은 양의 노이즈가 포함되어 있으므로 검사할 수 없습니다.

전문가 노이즈 제거기 앙상블로서 기본 모델은 고노이즈 확산 단계에서 전문가 역할을 하고 정제기 모델은 저노이즈 확산 단계에서 전문가 역할을 합니다. 기본 모델과 정제기 모델을 로드합니다.

In [ ]:
from diffusers import DiffusionPipeline
import torch

base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda")

이 접근 방식을 사용하려면 각 모델이 해당 단계를 실행할 시간 단계 수를 정의해야 합니다. 기본 모델의 경우 이는 [`denoising_end`](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLPipeline.__call__.denoising_end) 매개변수로 제어되고 정제기 모델의 경우 [`denoising_start`](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLImg2ImgPipeline.__call__.denoising_start) 매개변수로 제어됩니다.

<Tip>

`denoising_end` 및 `denoising_start` 매개변수는 0과 1 사이의 부동 소수점이어야 합니다. 이러한 매개변수는 스케줄러에 의해 정의된 불연속 시간 단계의 비율로 표시됩니다. `strength` 매개변수도 사용하는 경우 모델이 훈련된 불연속 시간 단계와 선언된 분수 차단에 의해 노이즈 제거 단계 수가 결정되므로 무시됩니다.

</Tip>

`denoising_end=0.8`로 설정하여 기본 모델이 **고노이즈** 시간 단계의 처음 80%를 노이즈 제거하도록 하고 `denoising_start=0.8`로 설정하여 정제기 모델이 **저노이즈** 시간 단계의 마지막 20%를 노이즈 제거하도록 합니다. 기본 모델 출력은 PIL 이미지가 아닌 **잠재** 공간이어야 합니다.

In [ ]:
prompt = "밤에 큰 돌에서 뛰어내리는 장엄한 사자"

image = base(
    prompt=prompt,
    num_inference_steps=40,
    denoising_end=0.8,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    num_inference_steps=40,
    denoising_start=0.8,
    image=image,
).images[0]
image

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lion_base.png" alt="밤에 바위 위에 있는 사자의 생성된 이미지" />
    <figcaption class="mt-2 text-center text-sm text-gray-500">기본 기본 모델</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lion_refined.png" alt="밤에 바위 위에 있는 사자의 고품질 생성 이미지" />
    <figcaption class="mt-2 text-center text-sm text-gray-500">전문가 노이즈 제거기 앙상블</figcaption>
  </div>
</div>

정제기 모델은 [StableDiffusionXLInpaintPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLInpaintPipeline)에서 인페인팅에도 사용할 수 있습니다.

In [ ]:
from diffusers import StableDiffusionXLInpaintPipeline
from diffusers.utils import load_image, make_image_grid
import torch

base = StableDiffusionXLInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

refiner = StableDiffusionXLInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda")

img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"

init_image = load_image(img_url)
mask_image = load_image(mask_url)

prompt = "벤치에 앉아 있는 장엄한 호랑이"
num_inference_steps = 75
high_noise_frac = 0.7

image = base(
    prompt=prompt,
    image=init_image,
    mask_image=mask_image,
    num_inference_steps=num_inference_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    image=image,
    mask_image=mask_image,
    num_inference_steps=num_inference_steps,
    denoising_start=high_noise_frac,
).images[0]
make_image_grid([init_image, mask_image, image.resize((512, 512))], rows=1, cols=3)

이 전문가 노이즈 제거기 앙상블 방법은 사용 가능한 모든 스케줄러에 대해 잘 작동합니다!

### 기본 모델에서 정제기 모델로

SDXL은 이미지-이미지 설정에서 기본 모델의 완전히 노이즈 제거된 이미지에 추가적인 고품질 세부 정보를 추가하기 위해 정제기 모델을 사용하여 이미지 품질을 향상시킵니다.

기본 모델과 정제기 모델을 로드합니다.

In [ ]:
from diffusers import DiffusionPipeline
import torch

base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda")

<Tip warning={true}>

SDXL 정제기를 다른 기본 모델과 함께 사용할 수 있습니다. 예를 들어 [Hunyuan-DiT](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../../api/pipelines/hunyuandit) 또는 [PixArt-Sigma](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../../api/pipelines/pixart_sigma) 파이프라인을 사용하여 프롬프트 준수도가 더 좋은 이미지를 생성할 수 있습니다. 이미지를 생성한 후에는 SDXL 정제기 모델에 전달하여 최종 생성 품질을 향상시킬 수 있습니다.

</Tip>

기본 모델에서 이미지를 생성하고 모델 출력을 **잠재** 공간으로 설정합니다.

In [ ]:
prompt = "정글 속 우주비행사, 차가운 색상 팔레트, 차분한 색상, 디테일, 8k"

image = base(prompt=prompt, output_type="latent").images[0]

생성된 이미지를 정제기 모델에 전달합니다.

In [ ]:
image = refiner(prompt=prompt, image=image[None, :]).images[0]

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/diffusers/docs-images/resolve/main/sd_xl/init_image.png" alt="화성에서 녹색 말을 타는 우주비행사의 생성된 이미지" />
    <figcaption class="mt-2 text-center text-sm text-gray-500">기본 모델</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/diffusers/docs-images/resolve/main/sd_xl/refined_image.png" alt="화성에서 녹색 말을 타는 우주비행사의 고품질 생성 이미지" />
    <figcaption class="mt-2 text-center text-sm text-gray-500">기본 모델 + 정제기 모델</figcaption>
  </div>
</div>

인페인팅의 경우 [StableDiffusionXLInpaintPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLInpaintPipeline)에서 기본 모델과 정제기 모델을 로드하고 `denoising_end` 및 `denoising_start` 매개변수를 제거하고 정제기에 대해 더 적은 추론 단계를 선택합니다.

## 마이크로 컨디셔닝

SDXL 훈련에는 *마이크로 컨디셔닝*이라고 하는 몇 가지 추가 컨디셔닝 기술이 포함됩니다. 여기에는 원본 이미지 크기, 대상 이미지 크기 및 자르기 매개변수가 포함됩니다. 마이크로 컨디셔닝은 추론 시 고품질의 중앙 정렬된 이미지를 만드는 데 사용할 수 있습니다.

<Tip>

분류기 없는 안내 덕분에 마이크로 컨디셔닝 및 음수 마이크로 컨디셔닝 매개변수를 모두 사용할 수 있습니다. 이는 [StableDiffusionXLPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLPipeline), [StableDiffusionXLImg2ImgPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLImg2ImgPipeline), [StableDiffusionXLInpaintPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLInpaintPipeline) 및 [StableDiffusionXLControlNetPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/controlnet_sdxl#diffusers.StableDiffusionXLControlNetPipeline)에서 사용할 수 있습니다.

</Tip>

### 크기 조절

크기 조절에는 두 가지 유형이 있습니다.

- [`original_size`](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLPipeline.__call__.original_size) 조절은 훈련 배치에서 확대된 이미지에서 비롯됩니다(전체 훈련 데이터의 거의 40%를 차지하는 작은 이미지를 버리는 것은 낭비이기 때문입니다). 이러한 방식으로 SDXL은 고해상도 이미지에 확대 아티팩트가 없어야 함을 학습합니다. 추론 중에 `original_size`를 사용하여 원본 이미지 해상도를 나타낼 수 있습니다. 기본값인 `(1024, 1024)`를 사용하면 데이터 세트의 1024x1024 이미지와 유사한 고품질 이미지가 생성됩니다. `(256, 256)`과 같은 낮은 해상도를 사용하도록 선택하면 모델은 여전히 1024x1024 이미지를 생성하지만 데이터 세트의 저해상도 이미지(더 간단한 패턴, 흐림)처럼 보입니다.

- [`target_size`](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLPipeline.__call__.target_size) 조절은 다양한 이미지 가로 세로 비율을 지원하도록 SDXL을 미세 조정하는 데서 비롯됩니다. 추론 중에 기본값인 `(1024, 1024)`를 사용하면 데이터 세트의 정사각형 이미지 구성과 유사한 이미지가 생성됩니다. `target_size`와 `original_size`에 동일한 값을 사용하는 것이 좋지만 다른 옵션도 자유롭게 실험해 보십시오!

🤗 Diffusers를 사용하면 특정 이미지 해상도에서 생성을 멀리하도록 이미지 크기에 대한 음수 조건을 지정할 수도 있습니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

prompt = "정글 속 우주비행사, 차가운 색상 팔레트, 차분한 색상, 디테일, 8k"
image = pipe(
    prompt=prompt,
    negative_original_size=(512, 512),
    negative_target_size=(1024, 1024),
).images[0]

<div class="flex flex-col justify-center">
  <img src="https://huggingface.co/datasets/diffusers/docs-images/resolve/main/sd_xl/negative_conditions.png"/>
  <figcaption class="text-center">이미지 해상도 (128, 128), (256, 256) 및 (512, 512)에 대해 음수 조건이 지정된 이미지입니다.</figcaption>
</div>

### 자르기 조절

이전 스테이블 디퓨전 모델에서 생성된 이미지는 때때로 잘린 것처럼 보일 수 있습니다. 이는 배치의 모든 이미지가 동일한 크기를 갖도록 훈련 중에 이미지가 실제로 잘리기 때문입니다. 자르기 좌표에 대한 조건을 지정함으로써 SDXL은 자르기 없음(좌표 `(0, 0)`)이 일반적으로 중앙에 있는 피사체 및 완전한 얼굴과 상관 관계가 있음을 *학습*합니다(이것이 🤗 Diffusers의 기본값입니다). 중앙에서 벗어난 구성을 생성하려면 다른 좌표를 실험해 볼 수 있습니다!

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

prompt = "정글 속 우주비행사, 차가운 색상 팔레트, 차분한 색상, 디테일, 8k"
image = pipeline(prompt=prompt, crops_coords_top_left=(256, 0)).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-cropped.png" alt="약간 잘린 정글 속 우주비행사의 생성된 이미지"/>
</div>

특정 자르기 매개변수에서 생성을 멀리하도록 음수 자르기 좌표를 지정할 수도 있습니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

prompt = "정글 속 우주비행사, 차가운 색상 팔레트, 차분한 색상, 디테일, 8k"
image = pipe(
    prompt=prompt,
    negative_original_size=(512, 512),
    negative_crops_coords_top_left=(0, 0),
    negative_target_size=(1024, 1024),
).images[0]
image

## 각 텍스트 인코더에 대해 다른 프롬프트 사용

SDXL은 두 개의 텍스트 인코더를 사용하므로 각 텍스트 인코더에 다른 프롬프트를 전달할 수 있으며, 이는 [품질을 향상](https://github.com/huggingface/diffusers/issues/4004#issuecomment-1627764201)시킬 수 있습니다. 원래 프롬프트를 `prompt`에 전달하고 두 번째 프롬프트를 `prompt_2`에 전달합니다(음수 프롬프트를 사용하는 경우 `negative_prompt` 및 `negative_prompt_2` 사용).

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

# 프롬프트는 OAI CLIP-ViT/L-14에 전달됩니다.
prompt = "정글 속 우주비행사, 차가운 색상 팔레트, 차분한 색상, 디테일, 8k"
# prompt_2는 OpenCLIP-ViT/bigG-14에 전달됩니다.
prompt_2 = "반 고흐 그림"
image = pipeline(prompt=prompt, prompt_2=prompt_2).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-double-prompt.png" alt="반 고흐 그림 스타일의 정글 속 우주비행사의 생성된 이미지"/>
</div>

이중 텍스트 인코더는 [SDXL 텍스트 반전](https://huggingface.co/docs/diffusers/main/en/using-diffusers/textual_inversion_inference#stable-diffusion-xl) 섹션에 설명된 대로 별도로 로드해야 하는 텍스트 반전 임베딩도 지원합니다.

## 최적화

SDXL은 대규모 모델이므로 하드웨어에서 실행하려면 메모리를 최적화해야 할 수 있습니다. 다음은 메모리를 절약하고 추론 속도를 높이는 몇 가지 팁입니다.

1. 메모리 부족 오류가 발생하면 [enable_model_cpu_offload()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.enable_model_cpu_offload)를 사용하여 모델을 CPU로 오프로드합니다.

```diff
- base.to("cuda")
- refiner.to("cuda")
+ base.enable_model_cpu_offload()
+ refiner.enable_model_cpu_offload()
```

2. `torch.compile`을 사용하여 약 20% 속도를 높입니다(`torch>=2.0` 필요).

```diff
+ base.unet = torch.compile(base.unet, mode="reduce-overhead", fullgraph=True)
+ refiner.unet = torch.compile(refiner.unet, mode="reduce-overhead", fullgraph=True)
```

3. `torch<2.0`인 경우 [xFormers](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/xformers)를 활성화하여 SDXL을 실행합니다.

```diff
+ base.enable_xformers_memory_efficient_attention()
+ refiner.enable_xformers_memory_efficient_attention()
```

## 기타 자료

SDXL에서 사용되는 [UNet2DConditionModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d-cond#diffusers.UNet2DConditionModel)의 최소 버전을 실험하는 데 관심이 있다면 PyTorch로 작성되었으며 🤗 Diffusers와 직접 호환되는 [minSDXL](https://github.com/cloneofsimo/minSDXL) 구현을 살펴보십시오.